# Uncapacitated Facility Location Problem

## Load the data

In [16]:
#### Loading the data
import os
import pandas as pd # type: ignore

# Parameters

#### food demand ####
food_per_person = 1.3 / 1000 # Tons per person

#### transportation costs ####
transportation_cost = 3364 / 10.5 # COP per ton per km

#### facility costs ####
### renting a warehouse
warehouse_cost_per_m2 = 24450 / 4 # COP/semana
### How much space is needed per ton of food
space_per_ton = 10.5/7.8 * 1.1 # ton per m2

# path to the data
data_path = 'data'
municipios_filename = 'municipios_procesado.csv'
distance_matrix_filename = 'distance_matrix_final.csv'
# load the data
population = pd.read_csv(os.path.join(data_path, municipios_filename),
                         usecols=['c_digo_dane_del_departamento', 'dpmp', '2024'])
distance_matrix = pd.read_csv(os.path.join(data_path, distance_matrix_filename),
                              index_col='dpmp')

# calculate the population per department = sum of the population of the municipalities in the department placed for each municipality in the department
population_per_department = population.groupby('c_digo_dane_del_departamento').sum()
population['population_per_department'] = population['c_digo_dane_del_departamento'].map(population_per_department['2024'])

In [ ]:
# Check for missing values
import missingno as msno
msno.matrix(population)


## Load the Model

In [17]:
import sys
sys.path.append("..")
from lp.uflp import UFLP

# h_i: demand of food in municipality i
h = population['2024'] * food_per_person
# c_ij: cost of transporting food from municipality i to j
c = distance_matrix * transportation_cost
# f_i: cost of building a warehouse in municipality i warehouse_cost_per_m2 * space_per_ton * department_demand
f = warehouse_cost_per_m2 * space_per_ton * population['population_per_department'] * food_per_person

# Turn the data to lists
h = h.tolist()
c = c.values.tolist()
f = f.tolist()
uflp = UFLP(h, c, f)
uflp.load_to_pulp()


## Solve the problem

In [23]:
#### Solving the problem
uflp.solve_by_pulp()

PulpSolverError: Pulp: Error while executing c:\Users\User\OneDrive\1_ Universidad\Trabajo de Grado\1. Proyecto de Grado\2. Expertimentación\AI_OR_Workflow\.venv\Lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [21]:
#### Saving the solution


## Display the data

In [22]:
#### Plotting the solution

## END